<a href="https://colab.research.google.com/github/testeclickpalm/Automacao-para-a-coleta-e-tratamento-de-dados-no-TABNET/blob/main/C%C3%B3pia_de_AUTOMA%C3%87%C3%83O_PARA_A_COLETA_E_O_TRATAMENTO_DE_DADOS_NO_TABNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automação do download das tabelas

In [1]:
'''
Versionamento:
Ambiente de execução hospedado do Colab
Tipo de ambiente de execução: Python 3
Acelerador de Hardware: CPU
'''

'''
A automação desenvolvida tem a função de facilitar a coleta e o tratamento de dados epidemiológicos do sistema TABNET, utilizado pelo DATASUS.
Ela automatiza o processo de busca, download, concatenação e correção de dados, eliminando a necessidade de etapas manuais demoradas e sujeitas a erros.
Utilizando Python e bibliotecas como Selenium e Pandas, a aplicação navega pelo site do TABNET, realiza o download das tabelas em formato CSV, padroniza
a formatação dos dados e corrige inconsistências, garantindo uma coleta de dados mais eficiente, precisa e atualizada para análise na área da saúde pública.
'''

# Baixa algumas tabelas que são de interesse:
# Exames por Ano segundo Faixa Etária - Local de Atendimento (Somente mulheres) - Cruz Alta, Giruá, Ijuí, Panambi - (2019 - 2024)
# Exames por Ano segundo Faixa Etária - Local de Residência (Somente mulheres) - Cruz Alta, Giruá, Ijuí, Panambi - (2019 - 2024)
# Exames por BI-RADS segundo Faixa Etária - Local de Atendimento (Somente mulheres) - Giruá e Panambi - (2019 - 2024)
# Exames por BI-RADS segundo Faixa Etária - Local de Residência (Somente mulheres) - Giruá e Panambi - (2019 - 2024)
# Exames por BI-RADS segundo Tempo Exame - Local de Atendimento (Somente mulheres) - Giruá e Panambi - (2019 - 2024)
# Exames por BI-RADS segundo Tempo Exame - Local de Residência (Somente mulheres) - Giruá e Panambi - (2019 - 2024)

# Concatena as todas as tabelas do mesmo conjunto
# Para baixá-las, é só rodar o código e abrir o diretório (ícone de pasta na esquerda) - Vai ter uma pasta chamada tabelas_concatenadas.

# Instala o pacote google-colab-selenium para automação web no Google Colab
%pip install -q google-colab-selenium

# Importações necessárias
import google_colab_selenium as gs  # Selenium para Google Colab
import os # Manipulação do sistema de arquivos (diretório)
import pandas as pd # Manipulação de dataframes
import time # Manipulação de tempo (para pausas)
from selenium import webdriver # Webdriver para automação de navegador
from selenium.webdriver.common.by import By # Localização de elementos
from selenium.webdriver.common.action_chains import ActionChains # Cadeias de ações no Selenium
from selenium.webdriver.common.keys import Keys # Simulação de teclas
from selenium.webdriver.support.ui import WebDriverWait # Esperas explícitas
from selenium.webdriver.support import expected_conditions as EC # Condições esperadas para esperas explícitas

# Diretório onde os arquivos serão salvos
download_dir = "/content"

# Classe para realizar a automação web
class WebScraper:
    def __init__(self):
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_experimental_option("prefs", {
            "download.default_directory": download_dir, # Define o diretório de download
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "safebrowsing.enabled": True
        })
        self.driver = gs.Chrome(options=chrome_options) # Inicia o navegador Chrome no Google Colab
        self.wait = WebDriverWait(self.driver, 10) # Configura espera explícita de 10 segundos
        self.driver.get('https://datasus.saude.gov.br/informacoes-de-saude-tabnet/')  # Abre a URL do Datasus

    def get_elemento(self, xpath):
        return self.wait.until(EC.presence_of_element_located((By.XPATH, xpath)))  # Espera até o elemento estar presente

    def clicar_elemento(self, xpath):
        elemento = self.get_elemento(xpath) # Localiza o elemento
        elemento.click() # Clica no elemento
        return elemento

    def escrever_no_campo(self, xpath, texto):
        campo = self.get_elemento(xpath) # Localiza o campo
        campo.send_keys(texto) # Escreve no campo
        return campo

    def limpar_campo(self, xpath):
        campo = self.get_elemento(xpath) # Localiza o campo
        campo.clear() # Limpa o campo

    def aguardar_visibilidade(self, xpath):
        return self.wait.until(EC.visibility_of_element_located((By.XPATH, xpath))) # Espera até o elemento estar visível

    def tirar_screenshot(self, nome_arquivo):
        self.driver.save_screenshot(nome_arquivo) # Salva uma captura de tela

    def selecionar_multiplas_opcoes(self, *xpaths):
        action = ActionChains(self.driver) # Inicia uma cadeia de ações
        action.key_down(Keys.CONTROL) # Pressiona a tecla CTRL
        for xpath in xpaths:
            self.aguardar_visibilidade(xpath) # Espera a visibilidade do elemento
            elemento = self.get_elemento(xpath) # Localiza o elemento
            action.click(elemento) # Clica no elemento
        action.key_up(Keys.CONTROL) # Solta a tecla CTRL
        action.perform() # Executa a cadeia de ações

    def trocar_para_nova_janela(self, janela_indice):
        nova_janela = self.driver.window_handles[janela_indice] # Obtém o identificador da nova janela
        self.driver.switch_to.window(nova_janela) # Troca para a nova janela

    def fechar_navegador(self):
        self.driver.quit() # Fecha o navegador

    def aguardar(self, segundos):
        time.sleep(segundos) # Aguarda um determinado número de segundos

    def verificar_download(self, download_xpath):
        # Obtém a lista de arquivos no diretório antes do download
        arquivos_antes = set(os.listdir(download_dir))
        tentativas = 0

        while True:
            try:
                # Clica no botão de download
                self.clicar_elemento(download_xpath)

                # Espera 5 segundos para o download iniciar
                self.aguardar(5)

                # Obtém a lista de arquivos no diretório após a tentativa de download
                arquivos_depois = set(os.listdir(download_dir))
                novos_arquivos = arquivos_depois - arquivos_antes

                if novos_arquivos:
                    return novos_arquivos.pop() # Retorna o nome do novo arquivo

                # Incrementa o contador de tentativas
                tentativas += 1

                # Limite de tentativas antes de tentar uma recuperação
                if tentativas % 10 == 0:
                    print("Tentativa de recuperação: Recarregando a página e tentando novamente.")
                    self.driver.refresh()  # Recarrega a página
                    self.aguardar(5)  # Espera para a página recarregar

            except Exception as e:
                print(f"Tentativa {tentativas + 1}: Erro ao tentar baixar o arquivo: {e}")
                # Espera 5 segundos antes de tentar novamente
                self.aguardar(5)

                # Incrementa o contador de tentativas
                tentativas += 1

                # Limite de tentativas antes de parar
                if tentativas >= 30:
                    raise Exception("Falha ao baixar o arquivo após várias tentativas.")


def faixaEtariaAnoAtendimento():
    try:
        # Inicializa o scraper
        scraper = WebScraper()

        # Começa no site do Tabnet

        # Clica em Epidemiológicas e Morbidade
        scraper.clicar_elemento('//*[@id="elementor-tab-title-2153"]')
        scraper.aguardar_visibilidade('//*[@id="elementor-tab-content-2153"]/ul/li[12]/span/a/span')

        # Clica Sistema de Informação do Câncer - SISCAN (colo do útero e mama)
        scraper.clicar_elemento('//*[@id="elementor-tab-content-2153"]/ul/li[12]/span/a/span')
        scraper.aguardar_visibilidade('//*[@id="infesq"]/input[8]')

        # Clica em Mamografia - Por local de atendimento
        scraper.clicar_elemento('//*[@id="infesq"]/input[8]')
        scraper.aguardar_visibilidade('//*[@id="mySelect"]/option[23]')

        # Clica em Rio Grande do Sul
        scraper.clicar_elemento('//*[@id="mySelect"]/option[23]')
        scraper.aguardar_visibilidade('//*[@id="L"]/option[7]')

        # Escolhe a opção de linha: Faixa etária
        scraper.clicar_elemento('//*[@id="L"]/option[7]')

        # Escolhe a opção de coluna: Ano de Competência
        scraper.clicar_elemento('//*[@id="C"]/option[2]')

        # Escolhe os últimos 6 anos, contando com o ano atual, que altera todo dia 20
        scraper.selecionar_multiplas_opcoes('//*[@id="A"]/option[2]', '//*[@id="A"]/option[3]', '//*[@id="A"]/option[4]', '//*[@id="A"]/option[5]', '//*[@id="A"]/option[6]')
        scraper.aguardar_visibilidade('//*[@id="fig2"]')

        # ------------ MUNICÍPIOS ------------
        # Cruz Alta: //*[@id="S2"]/option[127]
        # Giruá:     //*[@id="S2"]/option[183]
        # Ijuí:      //*[@id="S2"]/option[204]
        # Panambi:   //*[@id="S2"]/option[299]

        # Clica no filtro de município de prestação de serviço e escreve "Cruz Alta"
        scraper.clicar_elemento('//*[@id="fig2"]')
        scraper.clicar_elemento('//*[@id="S2"]/option[1]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes2"]', "Cruz Alta")
        scraper.aguardar_visibilidade('//*[@id="S2"]/option[127]')
        scraper.clicar_elemento('//*[@id="S2"]/option[127]')

        # Clica no filtro de sexo e marca a opção 'feminino'
        scraper.clicar_elemento('//*[@id="fig8"]')
        scraper.clicar_elemento('//*[@id="S8"]/option[1]')
        scraper.clicar_elemento('//*[@id="S8"]/option[3]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Clica no filtro de município de prestação de serviço e escreve "Giruá"
        scraper.clicar_elemento('//*[@id="S2"]/option[127]')
        scraper.limpar_campo('//*[@id="pesqmes2"]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes2"]', "Giruá")
        scraper.clicar_elemento('//*[@id="S2"]/option[183]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Clica no filtro de município de prestação de serviço e escreve "Ijuí"
        scraper.clicar_elemento('//*[@id="S2"]/option[183]')
        scraper.limpar_campo('//*[@id="pesqmes2"]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes2"]', "Ijuí")
        scraper.clicar_elemento('//*[@id="S2"]/option[204]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Clica no filtro de município de prestação de serviço e escreve "Panambi"
        scraper.clicar_elemento('//*[@id="S2"]/option[204]')
        scraper.limpar_campo('//*[@id="pesqmes2"]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes2"]', "Panambi")
        scraper.clicar_elemento('//*[@id="S2"]/option[299]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Trocar para a nova janela/aba (Cruz Alta)
        scraper.trocar_para_nova_janela(3)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá)
        scraper.trocar_para_nova_janela(2)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Ijuí)
        scraper.trocar_para_nova_janela(1)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi)
        scraper.trocar_para_nova_janela(4)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

    except Exception as e:
        print(f"Erro na função faixaEtariaAnoAtendimento: {e}")
    finally:
        scraper.fechar_navegador()


def faixaEtariaAnoResidencia():
    try:
        # Inicializa o scraper
        scraper = WebScraper()

        # Começa no site do Tabnet

        # Clica em Epidemiológicas e Morbidade
        scraper.clicar_elemento('//*[@id="elementor-tab-title-2153"]')
        scraper.aguardar_visibilidade('//*[@id="elementor-tab-content-2153"]/ul/li[12]/span/a/span')

        # Clica Sistema de Informação do Câncer - SISCAN (colo do útero e mama)
        scraper.clicar_elemento('//*[@id="elementor-tab-content-2153"]/ul/li[12]/span/a/span')
        scraper.aguardar_visibilidade('//*[@id="infesq"]/input[7]')

        # Clica em Mamografia - Por local de residência
        scraper.clicar_elemento('//*[@id="infesq"]/input[7]')
        scraper.aguardar_visibilidade('//*[@id="mySelect"]/option[23]')

        # Clica em Rio Grande do Sul
        scraper.clicar_elemento('//*[@id="mySelect"]/option[23]')
        scraper.aguardar_visibilidade('//*[@id="L"]/option[4]')

        # Escolhe a opção de linha: Faixa etária
        scraper.clicar_elemento('//*[@id="L"]/option[4]')

        # Escolhe a opção de coluna: Ano de Competência
        scraper.clicar_elemento('//*[@id="C"]/option[2]')

        # Escolhe os últimos 6 anos, contando com o ano atual, que altera todo dia 20
        scraper.selecionar_multiplas_opcoes('//*[@id="A"]/option[2]', '//*[@id="A"]/option[3]', '//*[@id="A"]/option[4]', '//*[@id="A"]/option[5]', '//*[@id="A"]/option[6]')
        scraper.aguardar_visibilidade('//*[@id="fig1"]')

        # ------------ MUNICÍPIOS ------------
        # Cruz Alta: //*[@id="S1"]/option[127]
        # Giruá:     //*[@id="S1"]/option[183]
        # Ijuí:      //*[@id="S1"]/option[204]
        # Panambi:   //*[@id="S1"]/option[299]

        # Clica no filtro de município de prestação de serviço e escreve "Cruz Alta"
        scraper.clicar_elemento('//*[@id="fig1"]')
        scraper.clicar_elemento('//*[@id="S1"]/option[1]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes1"]', "Cruz Alta")
        scraper.aguardar_visibilidade('//*[@id="S1"]/option[127]')
        scraper.clicar_elemento('//*[@id="S1"]/option[127]')

        # Clica no filtro de sexo e marca a opção 'feminino'
        scraper.clicar_elemento('//*[@id="fig6"]')
        scraper.clicar_elemento('//*[@id="S6"]/option[1]')
        scraper.clicar_elemento('//*[@id="S6"]/option[3]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Clica no filtro de município de prestação de serviço e escreve "Giruá"
        scraper.clicar_elemento('//*[@id="S1"]/option[127]')
        scraper.limpar_campo('//*[@id="pesqmes1"]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes1"]', "Giruá")
        scraper.clicar_elemento('//*[@id="S1"]/option[183]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Clica no filtro de município de prestação de serviço e escreve "Ijuí"
        scraper.clicar_elemento('//*[@id="S1"]/option[183]')
        scraper.limpar_campo('//*[@id="pesqmes1"]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes1"]', "Ijuí")
        scraper.clicar_elemento('//*[@id="S1"]/option[204]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Clica no filtro de município de prestação de serviço e escreve "Panambi"
        scraper.clicar_elemento('//*[@id="S1"]/option[204]')
        scraper.limpar_campo('//*[@id="pesqmes1"]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes1"]', "Panambi")
        scraper.clicar_elemento('//*[@id="S1"]/option[299]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Trocar para a nova janela/aba (Cruz Alta)
        scraper.trocar_para_nova_janela(3)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá)
        scraper.trocar_para_nova_janela(2)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Ijuí)
        scraper.trocar_para_nova_janela(1)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi)
        scraper.trocar_para_nova_janela(4)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

    except Exception as e:
        print(f"Erro na função faixaEtariaAnoResidencia: {e}")
    finally:
        scraper.fechar_navegador()


def faixaEtariaBiradsAtendimento():
    try:
        # Inicializa o scraper
        scraper = WebScraper()

        # Começa no site do Tabnet

        # Clica em Epidemiológicas e Morbidade
        scraper.clicar_elemento('//*[@id="elementor-tab-title-2153"]')
        scraper.aguardar_visibilidade('//*[@id="elementor-tab-content-2153"]/ul/li[12]/span/a/span')

        # Clica Sistema de Informação do Câncer - SISCAN (colo do útero e mama)
        scraper.clicar_elemento('//*[@id="elementor-tab-content-2153"]/ul/li[12]/span/a/span')
        scraper.aguardar_visibilidade('//*[@id="infesq"]/input[8]')

        # Clica em Mamografia - Por local de atendimento
        scraper.clicar_elemento('//*[@id="infesq"]/input[8]')
        scraper.aguardar_visibilidade('//*[@id="mySelect"]/option[23]')

        # Clica em Rio Grande do Sul
        scraper.clicar_elemento('//*[@id="mySelect"]/option[23]')
        scraper.aguardar_visibilidade('//*[@id="L"]/option[7]')

        # Escolhe a opção de linha: Faixa etária
        scraper.clicar_elemento('//*[@id="L"]/option[7]')

        # Escolhe a opção de coluna: BI-RADS
        scraper.clicar_elemento('//*[@id="C"]/option[19]')

        # Clica no filtro de sexo e marca a opção 'feminino'
        scraper.clicar_elemento('//*[@id="fig8"]')
        scraper.clicar_elemento('//*[@id="S8"]/option[1]')
        scraper.clicar_elemento('//*[@id="S8"]/option[3]')

        # ------------ MUNICÍPIOS ------------
        # Giruá:     //*[@id="S2"]/option[183]
        # Panambi:   //*[@id="S2"]/option[299]

        # Clica no filtro de município de prestação de serviço e escreve "Giruá"
        scraper.clicar_elemento('//*[@id="fig2"]')
        scraper.clicar_elemento('//*[@id="S2"]/option[1]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes2"]', "Giruá")
        scraper.clicar_elemento('//*[@id="S2"]/option[183]')

        # Escolhe o 6° ano
        scraper.clicar_elemento('//*[@id="A"]/option[1]')
        scraper.clicar_elemento('//*[@id="A"]/option[6]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 5° ano
        scraper.clicar_elemento('//*[@id="A"]/option[6]')
        scraper.clicar_elemento('//*[@id="A"]/option[5]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 4° ano
        scraper.clicar_elemento('//*[@id="A"]/option[5]')
        scraper.clicar_elemento('//*[@id="A"]/option[4]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 3° ano
        scraper.clicar_elemento('//*[@id="A"]/option[4]')
        scraper.clicar_elemento('//*[@id="A"]/option[3]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 2° ano
        scraper.clicar_elemento('//*[@id="A"]/option[3]')
        scraper.clicar_elemento('//*[@id="A"]/option[2]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 1° ano
        scraper.clicar_elemento('//*[@id="A"]/option[2]')
        scraper.clicar_elemento('//*[@id="A"]/option[1]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Clica no filtro de município de prestação de serviço e escreve "Panambi"
        scraper.clicar_elemento('//*[@id="S2"]/option[183]')
        scraper.limpar_campo('//*[@id="pesqmes2"]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes2"]', "Panambi")
        scraper.clicar_elemento('//*[@id="S2"]/option[299]')

        # Escolhe o 6° ano
        scraper.clicar_elemento('//*[@id="A"]/option[1]')
        scraper.clicar_elemento('//*[@id="A"]/option[6]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 4° ano
        scraper.clicar_elemento('//*[@id="A"]/option[6]')
        scraper.clicar_elemento('//*[@id="A"]/option[4]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 3° ano
        scraper.clicar_elemento('//*[@id="A"]/option[4]')
        scraper.clicar_elemento('//*[@id="A"]/option[3]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 2° ano
        scraper.clicar_elemento('//*[@id="A"]/option[3]')
        scraper.clicar_elemento('//*[@id="A"]/option[2]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 1° ano
        scraper.clicar_elemento('//*[@id="A"]/option[2]')
        scraper.clicar_elemento('//*[@id="A"]/option[1]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Trocar para a nova janela/aba (Giruá - 6° ano)
        scraper.trocar_para_nova_janela(9)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá - 5° ano)
        scraper.trocar_para_nova_janela(8)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá - 4° ano)
        scraper.trocar_para_nova_janela(7)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá - 3° ano)
        scraper.trocar_para_nova_janela(10)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá - 2° ano)
        scraper.trocar_para_nova_janela(6)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá - 1° ano)
        scraper.trocar_para_nova_janela(5)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 6° ano)
        scraper.trocar_para_nova_janela(4)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 4° ano)
        scraper.trocar_para_nova_janela(11)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 3° ano)
        scraper.trocar_para_nova_janela(3)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 2° ano)
        scraper.trocar_para_nova_janela(2)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 1° ano)
        scraper.trocar_para_nova_janela(1)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

    except Exception as e:
        print(f"Erro na função FaixaEtariaBiradsAtendimento: {e}")
    finally:
        scraper.fechar_navegador()


def faixaEtariaBiradsResidencia():
    try:
        # Inicializa o scraper
        scraper = WebScraper()

        # Começa no site do Tabnet

        # Clica em Epidemiológicas e Morbidade
        scraper.clicar_elemento('//*[@id="elementor-tab-title-2153"]')
        scraper.aguardar_visibilidade('//*[@id="elementor-tab-content-2153"]/ul/li[12]/span/a/span')

        # Clica Sistema de Informação do Câncer - SISCAN (colo do útero e mama)
        scraper.clicar_elemento('//*[@id="elementor-tab-content-2153"]/ul/li[12]/span/a/span')
        scraper.aguardar_visibilidade('//*[@id="infesq"]/input[7]')

        # Clica em Mamografia - Por local de residência
        scraper.clicar_elemento('//*[@id="infesq"]/input[7]')
        scraper.aguardar_visibilidade('//*[@id="mySelect"]/option[23]')

        # Clica em Rio Grande do Sul
        scraper.clicar_elemento('//*[@id="mySelect"]/option[23]')
        scraper.aguardar_visibilidade('//*[@id="L"]/option[4]')

        # Escolhe a opção de linha: Faixa etária
        scraper.clicar_elemento('//*[@id="L"]/option[4]')

        # Escolhe a opção de coluna: BI-RADS
        scraper.clicar_elemento('//*[@id="C"]/option[21]')

        # Clica no filtro de sexo e marca a opção 'feminino'
        scraper.clicar_elemento('//*[@id="fig6"]')
        scraper.clicar_elemento('//*[@id="S6"]/option[1]')
        scraper.clicar_elemento('//*[@id="S6"]/option[3]')

        # ------------ MUNICÍPIOS ------------
        # Giruá:     //*[@id="S1"]/option[183]
        # Panambi:   //*[@id="S1"]/option[299]

        # Clica no filtro de município de prestação de serviço e escreve "Giruá"
        scraper.clicar_elemento('//*[@id="fig1"]')
        scraper.clicar_elemento('//*[@id="S1"]/option[1]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes1"]', "Giruá")
        scraper.clicar_elemento('//*[@id="S1"]/option[183]')

        # Escolhe o 6° ano
        scraper.clicar_elemento('//*[@id="A"]/option[1]')
        scraper.clicar_elemento('//*[@id="A"]/option[6]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 5° ano
        scraper.clicar_elemento('//*[@id="A"]/option[6]')
        scraper.clicar_elemento('//*[@id="A"]/option[5]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 4° ano
        scraper.clicar_elemento('//*[@id="A"]/option[5]')
        scraper.clicar_elemento('//*[@id="A"]/option[4]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 3° ano
        scraper.clicar_elemento('//*[@id="A"]/option[4]')
        scraper.clicar_elemento('//*[@id="A"]/option[3]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 2° ano
        scraper.clicar_elemento('//*[@id="A"]/option[3]')
        scraper.clicar_elemento('//*[@id="A"]/option[2]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 1° ano
        scraper.clicar_elemento('//*[@id="A"]/option[2]')
        scraper.clicar_elemento('//*[@id="A"]/option[1]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Clica no filtro de município de prestação de serviço e escreve "Panambi"
        scraper.clicar_elemento('//*[@id="S1"]/option[183]')
        scraper.limpar_campo('//*[@id="pesqmes1"]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes1"]', "Panambi")
        scraper.clicar_elemento('//*[@id="S1"]/option[299]')

        # Escolhe o 6° ano
        scraper.clicar_elemento('//*[@id="A"]/option[1]')
        scraper.clicar_elemento('//*[@id="A"]/option[6]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 5° ano
        scraper.clicar_elemento('//*[@id="A"]/option[6]')
        scraper.clicar_elemento('//*[@id="A"]/option[5]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 4° ano
        scraper.clicar_elemento('//*[@id="A"]/option[5]')
        scraper.clicar_elemento('//*[@id="A"]/option[4]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 3° ano
        scraper.clicar_elemento('//*[@id="A"]/option[4]')
        scraper.clicar_elemento('//*[@id="A"]/option[3]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 2° ano
        scraper.clicar_elemento('//*[@id="A"]/option[3]')
        scraper.clicar_elemento('//*[@id="A"]/option[2]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 1° ano
        scraper.clicar_elemento('//*[@id="A"]/option[2]')
        scraper.clicar_elemento('//*[@id="A"]/option[1]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Trocar para a nova janela/aba (Giruá - 6° ano)
        scraper.trocar_para_nova_janela(10)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá - 5° ano)
        scraper.trocar_para_nova_janela(9)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá - 4° ano)
        scraper.trocar_para_nova_janela(8)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá - 3° ano)
        scraper.trocar_para_nova_janela(11)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá - 2° ano)
        scraper.trocar_para_nova_janela(7)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá - 1° ano)
        scraper.trocar_para_nova_janela(6)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 6° ano)
        scraper.trocar_para_nova_janela(5)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 5° ano)
        scraper.trocar_para_nova_janela(12)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 4° ano)
        scraper.trocar_para_nova_janela(4)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 3° ano)
        scraper.trocar_para_nova_janela(3)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 2° ano)
        scraper.trocar_para_nova_janela(2)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 1° ano)
        scraper.trocar_para_nova_janela(1)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

    except Exception as e:
        print(f"Erro na função FaixaEtariaBiradsAtendimento: {e}")
    finally:
        scraper.fechar_navegador()


def tempoExameBiradsAtendimento():
    try:
        # Inicializa o scraper
        scraper = WebScraper()

        # Começa no site do Tabnet

        # Clica em Epidemiológicas e Morbidade
        scraper.clicar_elemento('//*[@id="elementor-tab-title-2153"]')
        scraper.aguardar_visibilidade('//*[@id="elementor-tab-content-2153"]/ul/li[12]/span/a/span')

        # Clica Sistema de Informação do Câncer - SISCAN (colo do útero e mama)
        scraper.clicar_elemento('//*[@id="elementor-tab-content-2153"]/ul/li[12]/span/a/span')
        scraper.aguardar_visibilidade('//*[@id="infesq"]/input[8]')

        # Clica em Mamografia - Por local de atendimento
        scraper.clicar_elemento('//*[@id="infesq"]/input[8]')
        scraper.aguardar_visibilidade('//*[@id="mySelect"]/option[23]')

        # Clica em Rio Grande do Sul
        scraper.clicar_elemento('//*[@id="mySelect"]/option[23]')
        scraper.aguardar_visibilidade('//*[@id="L"]/option[25]')

        # Escolhe a opção de linha: Tempo Exame
        scraper.clicar_elemento('//*[@id="L"]/option[25]')

        # Escolhe a opção de coluna: BI-RADS
        scraper.clicar_elemento('//*[@id="C"]/option[19]')

        # Clica no filtro de sexo e marca a opção 'feminino'
        scraper.clicar_elemento('//*[@id="fig8"]')
        scraper.clicar_elemento('//*[@id="S8"]/option[1]')
        scraper.clicar_elemento('//*[@id="S8"]/option[3]')

        # ------------ MUNICÍPIOS ------------
        # Giruá:     //*[@id="S2"]/option[183]
        # Panambi:   //*[@id="S2"]/option[299]

        # Clica no filtro de município de prestação de serviço e escreve "Giruá"
        scraper.clicar_elemento('//*[@id="fig2"]')
        scraper.clicar_elemento('//*[@id="S2"]/option[1]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes2"]', "Giruá")
        scraper.clicar_elemento('//*[@id="S2"]/option[183]')

        # Escolhe o 6° ano
        scraper.clicar_elemento('//*[@id="A"]/option[1]')
        scraper.clicar_elemento('//*[@id="A"]/option[6]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 5° ano
        scraper.clicar_elemento('//*[@id="A"]/option[6]')
        scraper.clicar_elemento('//*[@id="A"]/option[5]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 4° ano
        scraper.clicar_elemento('//*[@id="A"]/option[5]')
        scraper.clicar_elemento('//*[@id="A"]/option[4]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 3° ano
        scraper.clicar_elemento('//*[@id="A"]/option[4]')
        scraper.clicar_elemento('//*[@id="A"]/option[3]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 2° ano
        scraper.clicar_elemento('//*[@id="A"]/option[3]')
        scraper.clicar_elemento('//*[@id="A"]/option[2]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 1° ano
        scraper.clicar_elemento('//*[@id="A"]/option[2]')
        scraper.clicar_elemento('//*[@id="A"]/option[1]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Clica no filtro de município de prestação de serviço e escreve "Panambi"
        scraper.clicar_elemento('//*[@id="S2"]/option[183]')
        scraper.limpar_campo('//*[@id="pesqmes2"]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes2"]', "Panambi")
        scraper.clicar_elemento('//*[@id="S2"]/option[299]')

        # Escolhe o 6° ano
        scraper.clicar_elemento('//*[@id="A"]/option[1]')
        scraper.clicar_elemento('//*[@id="A"]/option[6]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 4° ano
        scraper.clicar_elemento('//*[@id="A"]/option[6]')
        scraper.clicar_elemento('//*[@id="A"]/option[4]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 3° ano
        scraper.clicar_elemento('//*[@id="A"]/option[4]')
        scraper.clicar_elemento('//*[@id="A"]/option[3]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 2° ano
        scraper.clicar_elemento('//*[@id="A"]/option[3]')
        scraper.clicar_elemento('//*[@id="A"]/option[2]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 1° ano
        scraper.clicar_elemento('//*[@id="A"]/option[2]')
        scraper.clicar_elemento('//*[@id="A"]/option[1]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Trocar para a nova janela/aba (Giruá - 6° ano)
        scraper.trocar_para_nova_janela(9)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá - 5° ano)
        scraper.trocar_para_nova_janela(8)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá - 4° ano)
        scraper.trocar_para_nova_janela(7)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá - 3° ano)
        scraper.trocar_para_nova_janela(10)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá - 2° ano)
        scraper.trocar_para_nova_janela(6)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá - 1° ano)
        scraper.trocar_para_nova_janela(5)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 6° ano)
        scraper.trocar_para_nova_janela(4)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 4° ano)
        scraper.trocar_para_nova_janela(11)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 3° ano)
        scraper.trocar_para_nova_janela(3)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 2° ano)
        scraper.trocar_para_nova_janela(2)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 1° ano)
        scraper.trocar_para_nova_janela(1)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

    except Exception as e:
        print(f"Erro na função FaixaEtariaBiradsAtendimento: {e}")
    finally:
        scraper.fechar_navegador()


def tempoExameBiradsResidencia():
    try:
        # Inicializa o scraper
        scraper = WebScraper()

        # Começa no site do Tabnet

        # Clica em Epidemiológicas e Morbidade
        scraper.clicar_elemento('//*[@id="elementor-tab-title-2153"]')
        scraper.aguardar_visibilidade('//*[@id="elementor-tab-content-2153"]/ul/li[12]/span/a/span')

        # Clica Sistema de Informação do Câncer - SISCAN (colo do útero e mama)
        scraper.clicar_elemento('//*[@id="elementor-tab-content-2153"]/ul/li[12]/span/a/span')
        scraper.aguardar_visibilidade('//*[@id="infesq"]/input[7]')

        # Clica em Mamografia - Por local de residência
        scraper.clicar_elemento('//*[@id="infesq"]/input[7]')
        scraper.aguardar_visibilidade('//*[@id="mySelect"]/option[23]')

        # Clica em Rio Grande do Sul
        scraper.clicar_elemento('//*[@id="mySelect"]/option[23]')
        scraper.aguardar_visibilidade('//*[@id="L"]/option[24]')

        # Escolhe a opção de linha: Tempo Exame
        scraper.clicar_elemento('//*[@id="L"]/option[24]')

        # Escolhe a opção de coluna: BI-RADS
        scraper.clicar_elemento('//*[@id="C"]/option[21]')

        # Clica no filtro de sexo e marca a opção 'feminino'
        scraper.clicar_elemento('//*[@id="fig6"]')
        scraper.clicar_elemento('//*[@id="S6"]/option[1]')
        scraper.clicar_elemento('//*[@id="S6"]/option[3]')

        # ------------ MUNICÍPIOS ------------
        # Giruá:     //*[@id="S1"]/option[183]
        # Panambi:   //*[@id="S1"]/option[299]

        # Clica no filtro de município de prestação de serviço e escreve "Giruá"
        scraper.clicar_elemento('//*[@id="fig1"]')
        scraper.clicar_elemento('//*[@id="S1"]/option[1]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes1"]', "Giruá")
        scraper.clicar_elemento('//*[@id="S1"]/option[183]')

        # Escolhe o 6° ano
        scraper.clicar_elemento('//*[@id="A"]/option[1]')
        scraper.clicar_elemento('//*[@id="A"]/option[6]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 5° ano
        scraper.clicar_elemento('//*[@id="A"]/option[6]')
        scraper.clicar_elemento('//*[@id="A"]/option[5]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 4° ano
        scraper.clicar_elemento('//*[@id="A"]/option[5]')
        scraper.clicar_elemento('//*[@id="A"]/option[4]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 3° ano
        scraper.clicar_elemento('//*[@id="A"]/option[4]')
        scraper.clicar_elemento('//*[@id="A"]/option[3]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 2° ano
        scraper.clicar_elemento('//*[@id="A"]/option[3]')
        scraper.clicar_elemento('//*[@id="A"]/option[2]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 1° ano
        scraper.clicar_elemento('//*[@id="A"]/option[2]')
        scraper.clicar_elemento('//*[@id="A"]/option[1]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Clica no filtro de município de prestação de serviço e escreve "Panambi"
        scraper.clicar_elemento('//*[@id="S1"]/option[183]')
        scraper.limpar_campo('//*[@id="pesqmes1"]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes1"]', "Panambi")
        scraper.clicar_elemento('//*[@id="S1"]/option[299]')

        # Escolhe o 6° ano
        scraper.clicar_elemento('//*[@id="A"]/option[1]')
        scraper.clicar_elemento('//*[@id="A"]/option[6]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 5° ano
        scraper.clicar_elemento('//*[@id="A"]/option[6]')
        scraper.clicar_elemento('//*[@id="A"]/option[5]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 4° ano
        scraper.clicar_elemento('//*[@id="A"]/option[5]')
        scraper.clicar_elemento('//*[@id="A"]/option[4]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 3° ano
        scraper.clicar_elemento('//*[@id="A"]/option[4]')
        scraper.clicar_elemento('//*[@id="A"]/option[3]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 2° ano
        scraper.clicar_elemento('//*[@id="A"]/option[3]')
        scraper.clicar_elemento('//*[@id="A"]/option[2]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Escolhe o 1° ano
        scraper.clicar_elemento('//*[@id="A"]/option[2]')
        scraper.clicar_elemento('//*[@id="A"]/option[1]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Trocar para a nova janela/aba (Giruá - 6° ano)
        scraper.trocar_para_nova_janela(10)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá - 5° ano)
        scraper.trocar_para_nova_janela(9)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá - 4° ano)
        scraper.trocar_para_nova_janela(8)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá - 3° ano)
        scraper.trocar_para_nova_janela(11)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá - 2° ano)
        scraper.trocar_para_nova_janela(7)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Giruá - 1° ano)
        scraper.trocar_para_nova_janela(6)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 6° ano)
        scraper.trocar_para_nova_janela(5)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 5° ano)
        scraper.trocar_para_nova_janela(12)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 4° ano)
        scraper.trocar_para_nova_janela(4)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 3° ano)
        scraper.trocar_para_nova_janela(3)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 2° ano)
        scraper.trocar_para_nova_janela(2)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Trocar para a nova janela/aba (Panambi - 1° ano)
        scraper.trocar_para_nova_janela(1)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.verificar_download('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

    except Exception as e:
        print(f"Erro na função FaixaEtariaBiradsAtendimento: {e}")
    finally:
        scraper.fechar_navegador()


def limpar_diretorio():
    # Itera sobre todos os arquivos no diretório '/content'
    for arquivo in os.listdir('/content'):
        # Monta o caminho completo para o arquivo
        caminho_arquivo = os.path.join('/content', arquivo)
        try:
            # Verifica se o caminho corresponde a um arquivo e não a um diretório
            if os.path.isfile(caminho_arquivo):
                # Exclui o arquivo
                os.unlink(caminho_arquivo)
        except Exception as e:
            # Caso ocorra algum erro ao tentar excluir o arquivo, imprime o erro
            print(f"Erro ao excluir {caminho_arquivo}: {e}")

def nomear_arquivos():
    # Diretório onde os arquivos foram baixados
    diretorio = '/content'

    # Lista na ordem desejada
    ordem_desejada = [
        "cruzAltaFaixaEtariaAnoAtendimento",
        "giruaFaixaEtariaAnoAtendimento",
        "ijuiFaixaEtariaAnoAtendimento",
        "panambiFaixaEtariaAnoAtendimento",
        "cruzAltaFaixaEtariaAnoResidencia",
        "giruaFaixaEtariaAnoResidencia",
        "ijuiFaixaEtariaAnoResidencia",
        "panambiFaixaEtariaAnoResidencia",
        "girua2019FaixaEtariaBiradsAtendimento",
        "girua2020FaixaEtariaBiradsAtendimento",
        "girua2021FaixaEtariaBiradsAtendimento",
        "girua2022FaixaEtariaBiradsAtendimento",
        "girua2023FaixaEtariaBiradsAtendimento",
        "girua2024FaixaEtariaBiradsAtendimento",
        "panambi2019FaixaEtariaBiradsAtendimento",
        "panambi2021FaixaEtariaBiradsAtendimento",
        "panambi2022FaixaEtariaBiradsAtendimento",
        "panambi2023FaixaEtariaBiradsAtendimento",
        "panambi2024FaixaEtariaBiradsAtendimento",
        "girua2019FaixaEtariaBiradsResidencia",
        "girua2020FaixaEtariaBiradsResidencia",
        "girua2021FaixaEtariaBiradsResidencia",
        "girua2022FaixaEtariaBiradsResidencia",
        "girua2023FaixaEtariaBiradsResidencia",
        "girua2024FaixaEtariaBiradsResidencia",
        "panambi2019FaixaEtariaBiradsResidencia",
        "panambi2020FaixaEtariaBiradsResidencia",
        "panambi2021FaixaEtariaBiradsResidencia",
        "panambi2022FaixaEtariaBiradsResidencia",
        "panambi2023FaixaEtariaBiradsResidencia",
        "panambi2024FaixaEtariaBiradsResidencia",
        "girua2019TempoExameBiradsAtendimento",
        "girua2020TempoExameBiradsAtendimento",
        "girua2021TempoExameBiradsAtendimento",
        "girua2022TempoExameBiradsAtendimento",
        "girua2023TempoExameBiradsAtendimento",
        "girua2024TempoExameBiradsAtendimento",
        "panambi2019TempoExameBiradsAtendimento",
        "panambi2021TempoExameBiradsAtendimento",
        "panambi2022TempoExameBiradsAtendimento",
        "panambi2023TempoExameBiradsAtendimento",
        "panambi2024TempoExameBiradsAtendimento",
        "girua2019TempoExameBiradsResidencia",
        "girua2020TempoExameBiradsResidencia",
        "girua2021TempoExameBiradsResidencia",
        "girua2022TempoExameBiradsResidencia",
        "girua2023TempoExameBiradsResidencia",
        "girua2024TempoExameBiradsResidencia",
        "panambi2019TempoExameBiradsResidencia",
        "panambi2020TempoExameBiradsResidencia",
        "panambi2021TempoExameBiradsResidencia",
        "panambi2022TempoExameBiradsResidencia",
        "panambi2023TempoExameBiradsResidencia",
        "panambi2024TempoExameBiradsResidencia"
    ]

    # Lista os arquivos baixados no diretório
    arquivos_baixados = os.listdir(diretorio)

    # Filtra apenas os arquivos CSV baixados pelas funções atendimento e residencia
    arquivos_csv = [arquivo for arquivo in arquivos_baixados if arquivo.endswith('.csv')]

    # Ordena os arquivos pela data de modificação (do mais antigo ao mais recente)
    arquivos_csv.sort(key=lambda x: os.path.getmtime(os.path.join(diretorio, x)))

    # Verifica se o número de arquivos CSV corresponde ao número de nomes desejados
    if len(arquivos_csv) != len(ordem_desejada):
        raise ValueError("O número de arquivos CSV não corresponde ao número de nomes desejados")

    # Carrega os arquivos CSV na ordem desejada e atribui a variáveis individuais
    for nome_variavel, arquivo_csv in zip(ordem_desejada, arquivos_csv):
        caminho_arquivo = os.path.join(diretorio, arquivo_csv)
        # Ajuste o sep e o encoding conforme necessário
        globals()[nome_variavel] = pd.read_csv(caminho_arquivo, sep=';', encoding='unicode_escape')


# Função para mover a coluna 'Total' para o final
def coluna_total(df):
    columns = list(df.columns)
    coluna_total = [col for col in columns if 'total' in col.lower()]
    if coluna_total:
        for col in coluna_total:
            columns.append(columns.pop(columns.index(col)))
    return df[columns]


def concatenar():
    # Adicionando uma nova coluna de índice chamada 'Município' nas tabelas: Exames por Ano segundo Faixa Etária - Local de Atendimento e Exames por Ano segundo Faixa Etária - Local de Residência
    cruzAltaFaixaEtariaAnoAtendimento['Município'] = 'Cruz Alta'
    giruaFaixaEtariaAnoAtendimento['Município'] = 'Giruá'
    ijuiFaixaEtariaAnoAtendimento['Município'] = 'Ijuí'
    panambiFaixaEtariaAnoAtendimento['Município'] = 'Panambi'

    cruzAltaFaixaEtariaAnoResidencia['Município'] = 'Cruz Alta'
    giruaFaixaEtariaAnoResidencia['Município'] = 'Giruá'
    ijuiFaixaEtariaAnoResidencia['Município'] = 'Ijuí'
    panambiFaixaEtariaAnoResidencia['Município'] = 'Panambi'

    # Adicionando novas colunas de índice chamadas 'Município' e 'Ano' nas tabelas: Exames por BI-RADS segundo Faixa Etária - Local de Atendimento e Exames por BI-RADS segundo Faixa Etária - Local de Residência
    girua2019FaixaEtariaBiradsAtendimento['Município'] = 'Giruá'
    girua2020FaixaEtariaBiradsAtendimento['Município'] = 'Giruá'
    girua2021FaixaEtariaBiradsAtendimento['Município'] = 'Giruá'
    girua2022FaixaEtariaBiradsAtendimento['Município'] = 'Giruá'
    girua2023FaixaEtariaBiradsAtendimento['Município'] = 'Giruá'
    girua2024FaixaEtariaBiradsAtendimento['Município'] = 'Giruá'
    panambi2019FaixaEtariaBiradsAtendimento['Município'] = 'Panambi'
    panambi2021FaixaEtariaBiradsAtendimento['Município'] = 'Panambi'
    panambi2022FaixaEtariaBiradsAtendimento['Município'] = 'Panambi'
    panambi2023FaixaEtariaBiradsAtendimento['Município'] = 'Panambi'
    panambi2024FaixaEtariaBiradsAtendimento['Município'] = 'Panambi'

    girua2019FaixaEtariaBiradsResidencia['Município'] = 'Giruá'
    girua2020FaixaEtariaBiradsResidencia['Município'] = 'Giruá'
    girua2021FaixaEtariaBiradsResidencia['Município'] = 'Giruá'
    girua2022FaixaEtariaBiradsResidencia['Município'] = 'Giruá'
    girua2023FaixaEtariaBiradsResidencia['Município'] = 'Giruá'
    girua2024FaixaEtariaBiradsResidencia['Município'] = 'Giruá'
    panambi2019FaixaEtariaBiradsResidencia['Município'] = 'Panambi'
    panambi2020FaixaEtariaBiradsResidencia['Município'] = 'Panambi'
    panambi2021FaixaEtariaBiradsResidencia['Município'] = 'Panambi'
    panambi2022FaixaEtariaBiradsResidencia['Município'] = 'Panambi'
    panambi2023FaixaEtariaBiradsResidencia['Município'] = 'Panambi'
    panambi2024FaixaEtariaBiradsResidencia['Município'] = 'Panambi'

    girua2019FaixaEtariaBiradsAtendimento['Ano'] = '2019'
    girua2020FaixaEtariaBiradsAtendimento['Ano'] = '2020'
    girua2021FaixaEtariaBiradsAtendimento['Ano'] = '2021'
    girua2022FaixaEtariaBiradsAtendimento['Ano'] = '2022'
    girua2023FaixaEtariaBiradsAtendimento['Ano'] = '2023'
    girua2024FaixaEtariaBiradsAtendimento['Ano'] = '2024'
    panambi2019FaixaEtariaBiradsAtendimento['Ano'] = '2019'
    panambi2021FaixaEtariaBiradsAtendimento['Ano'] = '2021'
    panambi2022FaixaEtariaBiradsAtendimento['Ano'] = '2022'
    panambi2023FaixaEtariaBiradsAtendimento['Ano'] = '2023'
    panambi2024FaixaEtariaBiradsAtendimento['Ano'] = '2024'

    girua2019FaixaEtariaBiradsResidencia['Ano'] = '2019'
    girua2020FaixaEtariaBiradsResidencia['Ano'] = '2020'
    girua2021FaixaEtariaBiradsResidencia['Ano'] = '2021'
    girua2022FaixaEtariaBiradsResidencia['Ano'] = '2022'
    girua2023FaixaEtariaBiradsResidencia['Ano'] = '2023'
    girua2024FaixaEtariaBiradsResidencia['Ano'] = '2024'
    panambi2019FaixaEtariaBiradsResidencia['Ano'] = '2019'
    panambi2020FaixaEtariaBiradsResidencia['Ano'] = '2020'
    panambi2021FaixaEtariaBiradsResidencia['Ano'] = '2021'
    panambi2022FaixaEtariaBiradsResidencia['Ano'] = '2022'
    panambi2023FaixaEtariaBiradsResidencia['Ano'] = '2023'
    panambi2024FaixaEtariaBiradsResidencia['Ano'] = '2024'

    # Adicionando novas colunas de índice chamadas 'Município' e 'Ano' nas tabelas: Exames por BI-RADS segundo Tempo Exame - Local de Atendimento e Exames por BI-RADS segundo Tempo Exame - Local de Residência
    girua2019TempoExameBiradsAtendimento['Município'] = 'Giruá'
    girua2020TempoExameBiradsAtendimento['Município'] = 'Giruá'
    girua2021TempoExameBiradsAtendimento['Município'] = 'Giruá'
    girua2022TempoExameBiradsAtendimento['Município'] = 'Giruá'
    girua2023TempoExameBiradsAtendimento['Município'] = 'Giruá'
    girua2024TempoExameBiradsAtendimento['Município'] = 'Giruá'
    panambi2019TempoExameBiradsAtendimento['Município'] = 'Panambi'
    panambi2021TempoExameBiradsAtendimento['Município'] = 'Panambi'
    panambi2022TempoExameBiradsAtendimento['Município'] = 'Panambi'
    panambi2023TempoExameBiradsAtendimento['Município'] = 'Panambi'
    panambi2024TempoExameBiradsAtendimento['Município'] = 'Panambi'

    girua2019TempoExameBiradsResidencia['Município'] = 'Giruá'
    girua2020TempoExameBiradsResidencia['Município'] = 'Giruá'
    girua2021TempoExameBiradsResidencia['Município'] = 'Giruá'
    girua2022TempoExameBiradsResidencia['Município'] = 'Giruá'
    girua2023TempoExameBiradsResidencia['Município'] = 'Giruá'
    girua2024TempoExameBiradsResidencia['Município'] = 'Giruá'
    panambi2019TempoExameBiradsResidencia['Município'] = 'Panambi'
    panambi2020TempoExameBiradsResidencia['Município'] = 'Panambi'
    panambi2021TempoExameBiradsResidencia['Município'] = 'Panambi'
    panambi2022TempoExameBiradsResidencia['Município'] = 'Panambi'
    panambi2023TempoExameBiradsResidencia['Município'] = 'Panambi'
    panambi2024TempoExameBiradsResidencia['Município'] = 'Panambi'

    girua2019TempoExameBiradsAtendimento['Ano'] = '2019'
    girua2020TempoExameBiradsAtendimento['Ano'] = '2020'
    girua2021TempoExameBiradsAtendimento['Ano'] = '2021'
    girua2022TempoExameBiradsAtendimento['Ano'] = '2022'
    girua2023TempoExameBiradsAtendimento['Ano'] = '2023'
    girua2024TempoExameBiradsAtendimento['Ano'] = '2024'
    panambi2019TempoExameBiradsAtendimento['Ano'] = '2019'
    panambi2021TempoExameBiradsAtendimento['Ano'] = '2021'
    panambi2022TempoExameBiradsAtendimento['Ano'] = '2022'
    panambi2023TempoExameBiradsAtendimento['Ano'] = '2023'
    panambi2024TempoExameBiradsAtendimento['Ano'] = '2024'

    girua2019TempoExameBiradsResidencia['Ano'] = '2019'
    girua2020TempoExameBiradsResidencia['Ano'] = '2020'
    girua2021TempoExameBiradsResidencia['Ano'] = '2021'
    girua2022TempoExameBiradsResidencia['Ano'] = '2022'
    girua2023TempoExameBiradsResidencia['Ano'] = '2023'
    girua2024TempoExameBiradsResidencia['Ano'] = '2024'
    panambi2019TempoExameBiradsResidencia['Ano'] = '2019'
    panambi2020TempoExameBiradsResidencia['Ano'] = '2020'
    panambi2021TempoExameBiradsResidencia['Ano'] = '2021'
    panambi2022TempoExameBiradsResidencia['Ano'] = '2022'
    panambi2023TempoExameBiradsResidencia['Ano'] = '2023'
    panambi2024TempoExameBiradsResidencia['Ano'] = '2024'

    # Concatena tudo sem repetir o nome da coluna (Exames por Ano segundo Faixa Etária - Local de Atendimento)
    FaixaEtariaAnoAtendimento = pd.concat([cruzAltaFaixaEtariaAnoAtendimento, giruaFaixaEtariaAnoAtendimento, ijuiFaixaEtariaAnoAtendimento, panambiFaixaEtariaAnoAtendimento], ignore_index=False)
    FaixaEtariaAnoAtendimento.set_index(['Município', 'Faixa etária'], inplace=True)
    FaixaEtariaAnoAtendimento = coluna_total(FaixaEtariaAnoAtendimento)

    # Concatena tudo sem repetir o nome da coluna (Exames por Ano segundo Faixa Etária - Local de Residência)
    FaixaEtariaAnoResidencia = pd.concat([cruzAltaFaixaEtariaAnoResidencia, giruaFaixaEtariaAnoResidencia, ijuiFaixaEtariaAnoResidencia, panambiFaixaEtariaAnoResidencia], ignore_index=False)
    FaixaEtariaAnoResidencia.set_index(['Município', 'Faixa etária'], inplace=True)
    FaixaEtariaAnoResidencia = coluna_total(FaixaEtariaAnoResidencia)

    # Concatena tudo sem repetir o nome da coluna (Exames por BI-RADS segundo Faixa Etária - Local de Atendimento)
    FaixaEtariaBiradsAtendimento = pd.concat([girua2019FaixaEtariaBiradsAtendimento,
                                              girua2020FaixaEtariaBiradsAtendimento,
                                              girua2021FaixaEtariaBiradsAtendimento,
                                              girua2022FaixaEtariaBiradsAtendimento,
                                              girua2023FaixaEtariaBiradsAtendimento,
                                              girua2024FaixaEtariaBiradsAtendimento,
                                              panambi2019FaixaEtariaBiradsAtendimento,
                                              panambi2021FaixaEtariaBiradsAtendimento,
                                              panambi2022FaixaEtariaBiradsAtendimento,
                                              panambi2023FaixaEtariaBiradsAtendimento,
                                              panambi2024FaixaEtariaBiradsAtendimento], ignore_index=False)

    FaixaEtariaBiradsAtendimento.set_index(['Município', 'Ano', 'Faixa etária'], inplace=True)
    FaixaEtariaBiradsAtendimento = coluna_total(FaixaEtariaBiradsAtendimento)

    # Concatena tudo sem repetir o nome da coluna (Exames por BI-RADS segundo Faixa Etária - Local de Residência)
    FaixaEtariaBiradsResidencia = pd.concat([girua2019FaixaEtariaBiradsResidencia,
                                             girua2020FaixaEtariaBiradsResidencia,
                                             girua2021FaixaEtariaBiradsResidencia,
                                             girua2022FaixaEtariaBiradsResidencia,
                                             girua2023FaixaEtariaBiradsResidencia,
                                             girua2024FaixaEtariaBiradsResidencia,
                                             panambi2019FaixaEtariaBiradsResidencia,
                                             panambi2020FaixaEtariaBiradsResidencia,
                                             panambi2021FaixaEtariaBiradsResidencia,
                                             panambi2022FaixaEtariaBiradsResidencia,
                                             panambi2023FaixaEtariaBiradsResidencia,
                                             panambi2024FaixaEtariaBiradsResidencia], ignore_index=False)

    FaixaEtariaBiradsResidencia.set_index(['Município', 'Ano', 'Faixa etária'], inplace=True)
    FaixaEtariaBiradsResidencia = coluna_total(FaixaEtariaBiradsResidencia)

    # Concatena tudo sem repetir o nome da coluna (Exames por BI-RADS segundo Tempo Exame - Local de Atendimento)
    TempoExameBiradsAtendimento = pd.concat([girua2019TempoExameBiradsResidencia,
                                             girua2020TempoExameBiradsResidencia,
                                             girua2021TempoExameBiradsResidencia,
                                             girua2022TempoExameBiradsResidencia,
                                             girua2023TempoExameBiradsResidencia,
                                             girua2024TempoExameBiradsResidencia,
                                             panambi2019TempoExameBiradsResidencia,
                                             panambi2021TempoExameBiradsResidencia,
                                             panambi2022TempoExameBiradsResidencia,
                                             panambi2023TempoExameBiradsResidencia,
                                             panambi2024TempoExameBiradsResidencia], ignore_index=False)

    TempoExameBiradsAtendimento.set_index(['Município', 'Ano', 'Tempo Exame'], inplace=True)
    TempoExameBiradsAtendimento = coluna_total(TempoExameBiradsAtendimento)

    # Concatena tudo sem repetir o nome da coluna (Exames por BI-RADS segundo Tempo Exame - Local de Residência)
    TempoExameBiradsResidencia = pd.concat([girua2019TempoExameBiradsResidencia,
                                             girua2020TempoExameBiradsResidencia,
                                             girua2021TempoExameBiradsResidencia,
                                             girua2022TempoExameBiradsResidencia,
                                             girua2023TempoExameBiradsResidencia,
                                             girua2024TempoExameBiradsResidencia,
                                             panambi2019TempoExameBiradsResidencia,
                                             panambi2020TempoExameBiradsResidencia,
                                             panambi2021TempoExameBiradsResidencia,
                                             panambi2022TempoExameBiradsResidencia,
                                             panambi2023TempoExameBiradsResidencia,
                                             panambi2024TempoExameBiradsResidencia], ignore_index=False)

    TempoExameBiradsResidencia.set_index(['Município', 'Ano', 'Tempo Exame'], inplace=True)
    TempoExameBiradsResidencia = coluna_total(TempoExameBiradsResidencia)

    # Cria um diretório para salvar os arquivos se não existir
    tabelas_concatenadas = 'tabelas_concatenadas'
    if not os.path.exists(tabelas_concatenadas):
        os.makedirs(tabelas_concatenadas)

    # Salva os arquivos concatenados no diretório especificado
    FaixaEtariaAnoAtendimento.to_excel(os.path.join(tabelas_concatenadas, 'Exames_por_Ano_segundo_Faixa_Etária_Local_de_Atendimento.xlsx'))
    FaixaEtariaAnoResidencia.to_excel(os.path.join(tabelas_concatenadas, 'Exames_por_Ano_segundo_Faixa_Etária_Local_de_Residência.xlsx'))
    FaixaEtariaBiradsAtendimento.to_excel(os.path.join(tabelas_concatenadas, 'Exames_por_Birads_segundo_Faixa_Etária_Local_de_Atendimento.xlsx'))
    FaixaEtariaBiradsResidencia.to_excel(os.path.join(tabelas_concatenadas, 'Exames_por_Birads_segundo_Faixa_Etária_Local_de_Residência.xlsx'))
    TempoExameBiradsAtendimento.to_excel(os.path.join(tabelas_concatenadas, 'Exames_por_Birads_segundo_Tempo_Exame_Local_de_Atendimento.xlsx'))
    TempoExameBiradsResidencia.to_excel(os.path.join(tabelas_concatenadas, 'Exames_por_Birads_segundo_Tempo_Exame_Local_de_Residência.xlsx'))

    print(f'\nTabelas concatenadas disponíveis em: {os.path.abspath(tabelas_concatenadas)}\n')


# Main
if __name__ == "__main__":
    # Começa o código limpando o diretório para que não ocorra confusão em caso de múltiplos testes
    limpar_diretorio()
    # Exames por Ano segundo Faixa Etária - Local de Atendimento (Somente mulheres)
    faixaEtariaAnoAtendimento()
    # Exames por Ano segundo Faixa Etária - Local de Residência (Somente mulheres)
    faixaEtariaAnoResidencia()
    # Exames por BI-RADS segundo Faixa Etária - Local de Atendimento (Somente mulheres)
    faixaEtariaBiradsAtendimento()
    # Exames por BI-RADS segundo Faixa Etária - Local de Residência (Somente mulheres)
    faixaEtariaBiradsResidencia()
    # Exames por BI-RADS segundo Tempo Exame - Local de Atendimento (Somente mulheres)
    tempoExameBiradsAtendimento()
    # Exames por BI-RADS segundo Tempo Exame - Local de Residência (Somente mulheres)
    tempoExameBiradsResidencia()
    # Nomea os arquivos por município e local (atendimento ou residência)
    nomear_arquivos()
    # Concatena os quatro municípios em uma só tabela (atendimento e residência)
    concatenar()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.1 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Tabelas concatenadas disponíveis em: /content/tabelas_concatenadas



# Importar para o drive da ClickPalm

In [2]:
# Importar para o Drive da ClickPalm

from google.colab import drive
drive.mount('/content/drive')

import os
import shutil
from datetime import datetime

# Caminho local da pasta no Colab
caminho_local = '/content/tabelas_concatenadas'

# Gerar nome de pasta baseado na data e hora
data_atual = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
nome_pasta = f'tabelas_concatenadas_{data_atual}'

# Caminho no Google Drive onde você deseja salvar a pasta
caminho_no_drive = f'/content/drive/My Drive/Documentação/Análise/Câncer de mama/Dados Câncer de mama/{nome_pasta}/'

# Verificar se a pasta de destino existe
if not os.path.exists(caminho_no_drive):
    # Criar o diretório no Google Drive se não existir
    os.makedirs(caminho_no_drive)

# Copiar os arquivos para o Google Drive
for arquivo in os.listdir(caminho_local):
    caminho_arquivo_local = os.path.join(caminho_local, arquivo)
    caminho_arquivo_drive = os.path.join(caminho_no_drive, arquivo)

    # Verificar se é um arquivo e não uma pasta
    if os.path.isfile(caminho_arquivo_local):
        shutil.copy(caminho_arquivo_local, caminho_arquivo_drive)

print(f'\nArquivos copiados para {caminho_no_drive}')

Mounted at /content/drive

Arquivos copiados para /content/drive/My Drive/Documentação/Análise/Câncer de mama/Dados Câncer de mama/tabelas_concatenadas_2024-09-25_03-16-32/
